# PythonでのTeradata操作

## teradatamlライブラリ

### 接続

In [1]:
from teradataml import DataFrame, in_schema, create_context, remove_context, to_numeric

In [56]:
# 接続情報
con = create_context(host = "172.17.0.139",
                 username="suzukiuser",
                 password="suzukiuser")

/Users/localadmin/anaconda3/envs/suzuki/lib/python3.6/site-packages/teradataml/context/context.py:144: UserWarning: [Teradata][teradataml](TDML_2002) Overwriting an existing context associated with Teradata connection. Most of the operations on any teradataml DataFrames created before this will not work.
  warnings.warn(Messages.get_message(MessageCodes.OVERWRITE_CONTEXT))


In [57]:
df = DataFrame("CAN_For_Eval") # テーブル名から、teradatamlのDataFrameオブジェクトを作成。

In [4]:
df # 上から10が表示される (裏側ではselect top 10 ~が実行される)

          starttime       times feature     val
carid                                          
021    201603210631   12.443618   11405     0.0
021    201603210631   12.443618   11413  -200.0
021    201603210631   12.443618   11415     0.0
021    201603210631   12.443618   11417     0.0
021    201603210631   12.443618   11451     0.0
021    201603210631   12.443618   11463  -200.0
021    201603210631   12.443618   11433  823.75
021    201603210631   12.443618   11407     0.0
021    201603210631   12.443618   11403     0.0
021    201603210631   12.443618   11401     0.0

In [5]:
# DataFrameではindexを指定できる(指定しない場合は、プライマリーキー、キーがない場合は0,1,2,...が表示される)
df_2 = DataFrame("CAN_For_Eval", index_label="times")
df_2

           carid     starttime feature     val
times                                         
 12.443618   021  201603210631   11405     0.0
 12.443618   021  201603210631   11413  -200.0
 12.443618   021  201603210631   11415     0.0
 12.443618   021  201603210631   11417     0.0
 12.443618   021  201603210631   11451     0.0
 12.443618   021  201603210631   11463  -200.0
 12.443618   021  201603210631   11433  823.75
 12.443618   021  201603210631   11407     0.0
 12.443618   021  201603210631   11403     0.0
 12.443618   021  201603210631   11401     0.0

In [6]:
# クエリ実行結果からでもDataFrameが作れる。indexも指定できる
df_11451 = DataFrame.from_query("select * from CAN_103_486_CAR1V21_HDD_1 where feature = '11451'",
                    index_label="times")
df_11451

                         carid                 starttime               feature  val
times                                                                              
0.285236  021                   201604092247              11451                 0.0
1.506517  021                   201604091141              11451                 1.0
0.197348  021                   201604090709              11451                 1.0
1.603978  021                   201604082038              11451                 3.0
0.351087  021                   201604082315              11451                 2.0
1.495772  021                   201604111039              11451                 1.0
0.202406  021                   201604131741              11451                 0.0
1.488430  021                   201604190835              11451                 3.0
0.260497  021                   201604132125              11451                 2.0
0.217654  021                   201604150720              11451             

### データを見る

In [58]:
# 100万件のデータ
df_mini = DataFrame.from_query("select top 1000000 * from CAN_103_486_CAR1V21_HDD_1")

#### head

In [9]:
# head (大データでは極めて時間がかかるので、使うべきでない。100万件のデータで4秒程度。1000万件で40秒程度である。)
df_mini.head()

                  carid                 starttime     times               feature  val
0  021                   201604181325              2.467358  3A146                 0.0
1  021                   201604181325              2.467358  3A122                 0.0
2  021                   201604181325              2.467358  3A125                 0.0
3  021                   201604181325              2.467358  3A127                 0.0
4  021                   201604181325              2.467358  3A115                 0.0
5  021                   201604181325              2.467358  3A116                 0.0
6  021                   201604181325              2.467358  3A112                 0.0
7  021                   201604181325              2.467358  3A132                 0.0
8  021                   201604181325              2.467358  3A143                 0.0
9  021                   201604181325              2.467358  3A140                 0.0

#### tail

In [10]:
# tail (同じく大データでは極めて時間がかかるので、使うべきでない)
df_mini.tail()

                  carid                 starttime     times               feature  val
0  021                   201604082038              0.374021  11407                 0.0
1  021                   201604082038              0.374271  1AF51                 0.0
2  021                   201604082038              0.374271  1AF52                 0.0
3  021                   201604082038              0.374271  1AF53                 0.0
4  021                   201604082038              0.374271  1AF23                 0.0
5  021                   201604082038              0.374271  1AF25                 0.0
6  021                   201604082038              0.374271  1AF47                -8.0
7  021                   201604082038              0.374271  1AF67                 0.0
8  021                   201604082038              0.374021  11405                 0.0
9  021                   201604082038              0.374021  11401                 0.0

#### describe

In [11]:
# describe (100万件のデータで10秒くらい)。返り値はDataFrameではないので注意
df_mini.describe()

         times                  val
func                               
50%      1.476                    0
count  1000000              1000000
mean     1.661     2546314787195073
min        .16               -40960
max      5.561  8431864402344411000
75%      2.278                    1
25%        .86                    0
std      1.086   146501897523348300

In [12]:
# percentileを変更できる。デフォルトは25%, 50%, 75%
df_mini.describe(percentiles=[.1, .9])

         times                  val
func                               
min        .16               -40960
std       1.07   147227614489377100
10%       .432                    0
90%      2.775               14.902
max      5.561  8431864402344411000
mean     1.645   2571610660777561.7
count  1000000              1000000

In [13]:
# includeオプション。デフォルトではNoneで数値列だけについて計算するが、
# allとするとstring系の列についても、null以外のレコードのcountと、unique数を計算してくれる
df_mini.describe(percentiles=[.1, .9], include="all")

          carid starttime    times  feature                  val
func                                                            
10%        None      None     .408     None                    0
count   1000000   1000000  1000000  1000000              1000000
mean       None      None    1.659     None     2571610379302586
min        None      None     .167     None               -40960
max        None      None    5.561     None  8431864402344411000
90%        None      None    2.798     None                   15
std        None      None    1.083     None   147227598374208170
unique        1        65     None      549                 None

#### filter

In [ ]:
# filter
df_mini.filter()

#### select

In [14]:
# selectで文字列のリストを渡すと、列を選ぶことができる
df_mini.select(["feature", "val"])

                feature    val
0  12406                   0.0
1  12013                  -0.5
2  12016                   0.0
3  12017                   0.0
4  12035                   0.0
5  12037                   0.0
6  12033                  -0.5
7  12407                   0.0
8  12405                   0.0
9  12417                 751.0

#### indexing

In [ ]:
# []を使ったアクセス(Python内部的には、[]の中身を引数として、DataFrameインスタンスの__getitem__メソッドが呼ばれる)

In [ ]:
# ilocを使ったアクセス

In [ ]:
# locを使ったアクセス

#### sort

In [ ]:
# sort

### データを操作する

#### assign

teradatamlではassignでのみ、新しい列を作成できる

In [24]:
# assign: val4という列と、valtimeという列を作成
df_mini.assign(val4 = df_mini.val * 4, valtime = df_mini.val + df_mini.times)

                  carid                 starttime     times               feature  val  val4   valtime
0  021                   201604091832              0.286238  3A767                 0.0   0.0  0.286238
1  021                   201604091832              0.286238  3A754                 0.0   0.0  0.286238
2  021                   201604091832              0.286238  3A755                 0.0   0.0  0.286238
3  021                   201604091832              0.286238  3A756                 0.0   0.0  0.286238
4  021                   201604091832              0.286238  3A740                 0.0   0.0  0.286238
5  021                   201604091832              0.286238  3A741                 0.0   0.0  0.286238
6  021                   201604091832              0.286238  3A757                 0.0   0.0  0.286238
7  021                   201604091832              0.286238  3A752                 0.0   0.0  0.286238
8  021                   201604091832              0.286238  3A766       

計算は+, -, *, /, %がサポートされている。//(整数の商)や**(累乗)はサポートされていない。

In [34]:
# drop_columnsオプションをTrueにすると、以前あった列は落とされる
df_mini.assign(val4 = df_mini.val * 4, valtime = df_mini.val + df_mini.times,
               dummy1 = 1, dummy2 = "dummy",
              drop_columns=True)
# また、今回のように定数や文字列を入力することも可能

  dummy1 dummy2  val4  valtime
0      1  dummy   0.0  1.54316
1      1  dummy  12.0  4.54316
2      1  dummy   0.0  1.54316
3      1  dummy   0.0  1.54316
4      1  dummy   0.0  1.54316
5      1  dummy   0.0  1.54316
6      1  dummy   0.0  1.54316
7      1  dummy -32.0 -6.45684
8      1  dummy   0.0  1.54316
9      1  dummy   0.0  1.54316

#### drop

In [27]:
# drop(列)
df_mini.drop(["starttime", "carid"]) # 一つだけ落とすときは、リストでなくてもよい

                  carid                 starttime     times               feature  val
0  021                   201604092237              0.334673  3D063                 0.0
1  021                   201604092237              0.334673  3D066                 0.0
2  021                   201604092237              0.334673  3D052                 0.0
3  021                   201604092237              0.334673  3D055                 0.0
4  021                   201604092237              0.334673  3D045                 0.0
5  021                   201604092237              0.334673  3D046                 0.0
6  021                   201604092237              0.334673  3D057                 0.0
7  021                   201604092237              0.334673  3D064                 0.0
8  021                   201604092237              0.334673  3D062                 0.0
9  021                   201604092237              0.334673  3D073                 0.0

#### dropna

In [37]:
# dropna
df_mini.dropna() # 一つでもnullのある行を落とす

                  carid                 starttime     times               feature     val
0  021                   201604110824              1.555105  12907                   0.00
1  021                   201604110824              1.555353  11451                   0.00
2  021                   201604110824              1.555353  11433                 823.75
3  021                   201604110824              1.555353  11413                -200.00
4  021                   201604110824              1.555353  11417                   0.00
5  021                   201604110824              1.555353  11401                   0.00
6  021                   201604110824              1.555353  11415                   0.00
7  021                   201604110824              1.555353  11463                -200.00
8  021                   201604110824              1.555105  12906                   0.00
9  021                   201604110824              1.555105  12927                   0.00

In [59]:
# thresholdオプション
df_mini.dropna(thresh=2) # 2よりnullが多い行を落とす

                  carid                 starttime     times               feature          val
0  021                   201604092247              1.672069  12906                    0.000000
1  021                   201604092247              1.672271  13F47                   10.196078
2  021                   201604092247              1.672271  13F17                 2451.000000
3  021                   201604092247              1.672271  13F07                    0.000000
4  021                   201604092247              1.672390  1E304                    0.000000
5  021                   201604092247              1.672390  1E305                    0.000000
6  021                   201604092247              1.672390  1E303                    0.000000
7  021                   201604092247              1.672069  12907                    0.000000
8  021                   201604092247              1.672069  12905                   75.000000
9  021                   201604092247             

In [60]:
# any, allオプション
df_mini.dropna(how="all") # 全てnullの行を落とす

                  carid                 starttime     times               feature     val
0  021                   201604141023              1.632478  1B825                 308.75
1  021                   201604141023              1.632478  1B806                   0.00
2  021                   201604141023              1.632478  1B807                   0.00
3  021                   201604141023              1.632711  1EF61                  -0.25
4  021                   201604141023              1.632711  1EF64                   3.00
5  021                   201604141023              1.632711  1EF65                   0.00
6  021                   201604141023              1.632711  1EF62                   0.00
7  021                   201604141023              1.632478  1B827                   0.00
8  021                   201604141023              1.632478  1B847                   0.00
9  021                   201604141023              1.632478  1B867                   0.00

In [61]:
# subsetオプション
df_mini.dropna(how="all", subset=["carid", "starttime"]) # subsetで選択した列について、全てnullの行を落とす

                  carid                 starttime     times               feature   val
0  021                   201604110824              1.564714  12016                  0.0
1  021                   201604110824              1.564714  12035                  0.0
2  021                   201604110824              1.564714  12037                  0.0
3  021                   201604110824              1.564714  12051                  2.0
4  021                   201604110824              1.564714  12054                  0.0
5  021                   201604110824              1.564714  12057                  0.0
6  021                   201604110824              1.564714  12052                  0.0
7  021                   201604110824              1.564714  12017                  0.0
8  021                   201604110824              1.564714  12013                 34.5
9  021                   201604110824              1.564513  12407                  0.0

#### 集約(count/max/min/mean/std/sum)

In [38]:
# count max, min, mean, std, sum
df_mini.max() # min, countも同様

              max_carid             max_starttime  max_times           max_feature       max_val
0  021                   201604192042               5.560586  50007                 8.431864e+18

In [41]:
df_mini.mean() # std, sumも同様。数値系の列のみ計算される

   mean_times      mean_val
0    1.704087  2.563178e+15

#### groupby

In [42]:
# groupby
df_mini.groupby("feature").min() # std, sum, count, min, maxも同様

                feature             min_carid             min_starttime  min_times  min_val
0  3D077                 021                   201604082038               0.182768      0.0
1  3D052                 021                   201604082038               0.182768      0.0
2  3B855                 021                   201604082038               0.175314      1.0
3  3A125                 021                   201604082038               0.162376      0.0
4  3D945                 021                   201604082038               0.183437      0.0
5  3D025                 021                   201604082038               0.182768      0.0
6  3C112                 021                   201604082038               0.193587      0.0
7  3A717                 021                   201604082038               0.201973      0.0
8  3D573                 021                   201604082038               0.193538      0.0
9  3D105                 021                   201604082038               0.1927

In [43]:
# groupby(複数でgroupする場合)
df_mini.groupby(["starttime","feature"]).min() # std, sum, count, min, maxも同様

                feature                 starttime             min_carid  min_times  min_val
0  3E212                 201604092202              021                    0.334040    1.000
1  3D541                 201604181204              021                    0.217951    0.000
2  31061                 201604092247              021                    0.352561    0.000
3  3C313                 201604091832              021                    0.294092    0.000
4  31457                 201604120829              021                    1.638601    0.445
5  32E33                 201604091203              021                    0.264973    0.000
6  1AF01                 201604092247              021                    0.285486    0.000
7  31802                 201604142234              021                    0.370648    0.000
8  31827                 201604111736              021                    0.295908  588.000
9  3C377                 201604130633              021                    0.2086

#### agg

In [45]:
# agg
df_mini.groupby(["starttime","feature"]).agg({"val":["mean","max"], "times":["std"]})

                feature                 starttime    mean_val  max_val  std_times
0  2A216                 201604091208                0.000000    0.000   2.009458
1  3D667                 201604131335                5.000000    5.000        NaN
2  3E212                 201604092202                1.000000    1.000   0.302254
3  47F06                 201604110931                1.000000    1.000        NaN
4  31827                 201604111736              588.000000  588.000   0.736644
5  3A702                 201604090732                0.000000    0.000   0.216026
6  31802                 201604142234                0.000000    0.000   0.405548
7  31061                 201604092247                0.000000    0.000   0.418275
8  31457                 201604120829                0.453333    0.455   0.375260
9  3D603                 201604110941                0.000000    0.000   0.389497

In [49]:
# agg(特に列を指定しない場合はlistで入れる。数値列のみ返るので注意)
df_mini.groupby(["starttime","feature"]).agg(["max","min"])

                feature                 starttime             max_carid             min_carid  max_times  min_times  max_val  min_val
0  3D541                 201604181204              021                   021                    0.217951   0.217951      0.0      0.0
1  3E212                 201604092202              021                   021                    1.529876   0.631396      1.0      1.0
2  3D667                 201604131335              021                   021                    0.216893   0.216893      5.0      5.0
3  3C377                 201604130633              021                   021                    0.208667   0.208667      0.0      0.0
4  31827                 201604111736              021                   021                    2.795961   0.295908    588.0    588.0
5  3E237                 201604082048              021                   021                    2.886065   0.590623      0.0      0.0
6  3A702                 201604090732              021        

In [51]:
# aggはgroupbyなしでも使える
df_mini.agg({"val":["min","max","std"]})

   min_val       max_val       std_val
0 -40960.0  8.431864e+18  1.479495e+17

In [53]:
# aggでは"unique"もつかえる
df_mini.agg({"val":["unique"]})

  unique_val
0      18324

In [44]:
# describeも使える。この場合はDataFrameでは返ってこないので注意
df_mini.groupby("feature").describe()

                            times    val
feature              func               
0FA21                25%     .842      0
                     50%    1.453      0
                     75%    2.283      0
                     count   6092   6092
                     max    5.552      0
                     mean   1.641      0
                     min     .161      0
                     std    1.083      0
0FA22                25%     .842      0
                     50%    1.453      0
                     75%    2.282      0
                     count   6091   6091
                     max    5.552      0
                     mean   1.641      0
                     min     .161      0
                     std    1.082      0
0FA23                25%     .842      0
                     50%    1.453      0
                     75%    2.282      0
                     count   6091   6091
                     max    5.552      0
                     mean   1.641      0
                

#### join

In [ ]:
# join


pandas同等、"="のjoinのみサポートされており、不等号joinはサポートされていないので注意。

#### merge

joinは同じ名前の列がないと結合できないが、mergeはそれぞれのテーブルで結合に使う列を指定できる

In [ ]:
# merge


#### indexの指定

In [ ]:
# set_index


### Pandas ⇔ Teradata

#### to_Pandas

#### copy_to_sql